In [1]:
import pandas as pd
import numpy as np
import sqlite3
import matplotlib.pyplot as plt
import os
from datetime import datetime as dt
import datetime

In [2]:
#DEFINE FUNÇÕES PRINCIPAIS PARA ACESSO DO BANCO DE DADOS COM A BASE INICIAL

conn = sqlite3.connect('DIST_DB_ANEEL.db')
cursor = conn.cursor()
conn_bi = sqlite3.connect('BD_POWER_BI_FINAL.db')
cursor_bi = conn_bi.cursor()

In [3]:
#PREPARA TABELA COM INFORMAÇÕES UTEIS PARA ANALISE

tableName = "DIST_INT_ELE_CONSOLIDADO"
columnsNecessary = f'{tableName}.SigAgente, {tableName}.FatGeradorInterrupcao, {tableName}.DscTipoInterrupcao, \
                     {tableName}.IdeMotivoExpurgo, {tableName}.DatInicioInterrupcao, {tableName}.DatFimInterrupcao, {tableName}.NumNivelTensao, {tableName}.NumUnidadeConsumidora, \
                     {tableName}.NumAno'

query = f""" SELECT {columnsNecessary}
             FROM {tableName}
             WHERE SigAgente LIKE '%ENEL CE%'    
             
"""

df = pd.read_sql_query(query,conn)


In [4]:
def changeTensaoToGroup(tensao): #Muda tensão para grupo
    if tensao == 13800:
        return "A"
    elif tensao == 220:
        return "B"
    elif tensao == 69000:
        return "A"


df_modificada = df.copy()
df_modificada['NumNivelTensao'] = df_modificada['NumNivelTensao'].apply(lambda x: changeTensaoToGroup(x))
df_modificada.rename(columns={'NumNivelTensao':'GrupoConsumidor'}, inplace=True)
df_modificada['DuracaoInterrupcaoHoras'] = ''
df_modificada['DatInicioInterrupcao'] = pd.to_datetime(df_modificada['DatInicioInterrupcao'])
df_modificada['DatFimInterrupcao'] = pd.to_datetime(df_modificada['DatFimInterrupcao'])

#Calcula a duração em horas de cada interrupção

for row in df_modificada.itertuples():
    inicio = row.DatInicioInterrupcao
    fim = row.DatFimInterrupcao
    duracao = round(float(((fim - inicio).total_seconds())/3600),2)
    df_modificada.at[row.Index,'DuracaoInterrupcaoHoras'] = duracao

df_modificada['TempoInterrupcaoGlobal'] = df_modificada['NumUnidadeConsumidora']*df_modificada['DuracaoInterrupcaoHoras']
    

# for gp in set(df_modificada['GrupoConsumidor']):
#     df_temp = df_modificada[df_modificada['GrupoConsumidor']==gp]
#     df_temp.to_csv(rf"C:\Users\marco\Desktop\teste_parcial_{gp}.csv", sep=';',index=False, encoding='ansi', decimal=',')

In [5]:
#Agrupa por ano e soma a duração de interrupção por tipo (programada/não-programada)

df_final_bi = df_modificada.copy()
df_final_bi = df_final_bi[['SigAgente','DscTipoInterrupcao','NumUnidadeConsumidora','GrupoConsumidor','NumAno','DuracaoInterrupcaoHoras','DatInicioInterrupcao','TempoInterrupcaoGlobal']]
df_final_bi.to_sql('InterrupcaoEletricidadePorGrupo',conn_bi, if_exists='replace', index=False)

conn.close()
conn_bi.close()